In [1]:
# install dependencies
import sys
!{sys.executable} -m pip install requests pandas plotly

In [2]:
from datetime import datetime, timedelta

'''
Generates a list of date ranges in the format "<start_date>:<end_date>"
for the past <num_of_weeks> weeks.

This format is required by the npm downloads API
(e.g., /downloads/range/2023-02-07:2023-02-14 returns daily downloads for that week.)
'''
def generate_weeks_range(num_weeks):
    current_date = datetime.today()
    weeks_range = []

    for i in range(num_weeks, 0, -1):
        week_start = current_date - timedelta(weeks=i)
        week_end = week_start + timedelta(days=6)
        # format it as "<start_date>:<end_date>"
        weeks_range.append(f"{week_start.date().strftime('%Y-%m-%d')}:{week_end.date().strftime('%Y-%m-%d')}")

    return weeks_range

In [3]:
import requests

'''
Retrieves per-day download stats from the npm API for a given package
within a list of date ranges formatted as "<start_date>:<end_date>" of a week (the parameter weeks_range).

Week ranges are used because the npm downloads endpoint only supports retrieving daily download stats
within either a week or a month. Using week ranges provides better granularity than month-based queries.

The result is a list of daily download entries in the format:
{"downloads": <number>, "day": <yyyy-mm-dd>}
'''
def get_daily_downloads_from_npm(package_name, weeks_range):
  # holds all daily download stats across the specified date ranges
  daily_downloads = []

  for week_start_and_end in weeks_range:
    npm_request_url = f"https://api.npmjs.org/downloads/range/{week_start_and_end}/{package_name}"
    response_json = requests.get(npm_request_url).json()

    # check to make sure the download data is present in the response
    if "downloads" not in response_json:
      raise Exception(f"download statistics for package not found in week {week_start_and_end}")

    # get the downaload stats from the response
    downloads_per_day_curr_week = response_json["downloads"]
    daily_downloads.extend(downloads_per_day_curr_week)

  return daily_downloads

In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

'''
Plot the daily download stats using Plotly
'''
def plot_download_stats(downloads_per_day_last_weeks, package_name, num_of_weeks):
  downloads_df = pd.DataFrame(downloads_per_day_last_weeks)
  # convert 'day' column to datetime
  downloads_df["day"] = pd.to_datetime(downloads_df["day"])
  # sort by date
  downloads_df = downloads_df.sort_values("day")

  # create Plotly figure
  fig = make_subplots(specs=[[{"secondary_y": False}]])

  fig.add_trace(go.Scatter(
      x=downloads_df["day"],
      y=downloads_df["downloads"],
      name='Daily Downloads',
      mode='lines+markers',
      line=dict(color='blue'),
      marker=dict(size=6)
  ))

  fig.update_layout(
      width=1000,
      height=500,
      paper_bgcolor='white',
      plot_bgcolor='#fafafa',
      hovermode='closest',
      title=f"Daily downloads of {package_name} in the last {num_of_weeks} weeks",
      xaxis=dict(
          title="Date",
          tickangle=45,
          type='date'
      ),
      yaxis=dict(
          title="Downloads"
      ),
      showlegend=False
  )

  fig.show()

In [5]:
'''
The full workflow: genate ranges, fetch data, and visualize
'''
def visualize_daily_downloads_in_past_weeks(package_name, num_of_weeks):
  weeks_range = generate_weeks_range(num_of_weeks)
  download_stats = get_daily_downloads_from_npm(package_name, weeks_range)
  # preview the data format
  print(f"download stats preview:\n {download_stats[-10:]}")
  plot_download_stats(download_stats, package_name, num_of_weeks)

In [6]:
# run the application/flow for a example package
# the package is published 3 months ago, so downloads prior to that will be 0
package_name = "@heroui/dom-animation"
num_of_weeks = 20
visualize_daily_downloads_in_past_weeks(package_name, num_of_weeks)

download stats preview:
 [{'downloads': 7522, 'day': '2025-04-13'}, {'downloads': 18549, 'day': '2025-04-14'}, {'downloads': 25463, 'day': '2025-04-15'}, {'downloads': 20860, 'day': '2025-04-16'}, {'downloads': 19821, 'day': '2025-04-17'}, {'downloads': 16460, 'day': '2025-04-18'}, {'downloads': 6981, 'day': '2025-04-19'}, {'downloads': 6941, 'day': '2025-04-20'}, {'downloads': 14497, 'day': '2025-04-21'}, {'downloads': 0, 'day': '2025-04-22'}]
